In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import Dict, List, Set, Callable, Optional

In [2]:
#seed 고정
import random
import os

def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED']=str(seed)
  np.random.seed(seed)

seed_everything(42)

### 데이터 로드

In [25]:
food_review=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\KoBERT 감성분류\data\food_review.csv')
kakao=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\KoBERT 감성분류\data\kakao_review_0to500.csv',encoding='cp949')
thoure=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\KoBERT 감성분류\data\1500개_리뷰_label.CSV',encoding='cp949')
hong=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\KoBERT 감성분류\data\홍대_맛집_리뷰_데이터_ver5.csv')

In [105]:
rr=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\KoBERT 감성분류\data\restaurant_review.txt',sep='\t')
rr.head()

,4,맛있는데 고기 양이 조금 적을 수도.
0,4.0,점심시간만되면 문전성시인곳. 혼자와서 먹는사람도 많음. 진짜 꼬수운 청국장 생각날때...
1,4.5,맛있어요
2,5.0,.
3,4.5,반찬이 부실하네요
4,4.0,맛있어요


### 전처리

In [26]:
food_review.drop('score',axis=1,inplace=True)
kakao.drop(['store_id','kakao_nickname','kakao_date'],axis=1,inplace=True)
thoure=thoure[['review','label']]
hong=hong[['user_rating','comment']]

In [30]:
display(food_review.head())
display(kakao.head())
display(thoure.head())
display(hong.head())

,review,y
0,친절하시고 깔끔하고 좋았습니다,1
1,조용하고 고기도 굿,1
2,"갈비탕과 냉면, 육회비빔밥이 맛있습니다.",1
3,대체적으로 만족하나\n와인의 구성이 살짝 아쉬움,1
4,고기도 맛있고 서비스는 더 최고입니다~,1


,kakao_content,kakao_star
0,차돌쭈꾸미 새로생겨서 먹어봤는디 정말맛있어요존맛탱구리구리 오늘도 아주 잘먹구갑니다 ...,5.0
1,진짜 맛집 근처 다른 해물찜들 먹다가 가게가 보여서 들어와 봤는데 진짜 맛있네요벌써...,5.0
2,처음 방문했는데 생각보다 많이 달라서 당황했습니다 천안본점에는 밑반찬만 6가지 정도...,1.0
3,지금껏 먹어본 해물찜과는 정말 다르게 너무 맛있어서 먹고 포장까지 해왔습니다새우찜도...,5.0
4,생고기 맛 좋아서 추천합니다,5.0


,review,label
0,주차시설좋고 넓고 깔끔하고 좋아요. 집 근처라 더욱 좋네요,1
1,아포카토 넘넘 맛있어요??,1
2,넓고 쾌적한 실내가 좋아요. 커피도 굿,1
3,매장이 넓어서 좋아요!,1
4,친절하시고 음료도 맛있어요 :),1


,user_rating,comment
0,5.0,NaN
1,5.0,❤️❤️❤️❤️❤️
2,5.0,오래 해주세요🙏
3,3.0,웨이팅해서 먹을 맛은 아니에요… 닭보쌈은 보통이었고 비빔국수라고 해서 당연히 국물 ...
4,5.0,와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!


In [34]:
#null값 없애기
kakao.dropna(axis=0,inplace=True)
hong.dropna(axis=0,inplace=True)

In [76]:
display(food_review.info())
display(kakao.info())
display(thoure.info())
display(hong.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  545 non-null    object
 1   label   545 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.6+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 965 entries, 0 to 964
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   kakao_content  965 non-null    object 
 1   label          965 non-null    float64
dtypes: float64(1), object(1)
memory usage: 15.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1574 entries, 0 to 1573
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  1574 non-null   object
 1   label   1574 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.7+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  2201 non-null   object 
 1   label    2201 non-null   float64
dtypes: float64(1), object(1)
memory usage: 34.5+ KB


None

In [110]:
#5,4는 1의 긍정레이블, 1,2는 부정레이블, 3은 삭제
def label_mapping(labels):
    return [1 if label in [5, 4, 4.5] else 0 if label in [1, 2,0.5,2.5,1.5] else label for label in labels]


In [53]:
kakao_new_labels=label_mapping(kakao['kakao_star'])
new_labels=label_mapping(hong['user_rating'])
rr_labels=label_mapping(rr['4'])

In [54]:
kakao['label']=kakao_new_labels
hong['label']=new_labels
rr['label']=rr_labels

In [62]:
hong3=hong[hong['label']==3].index.tolist()
hong.drop(hong3,axis=0,inplace=True)
hong=hong.reset_index(drop=True)


,user_rating,comment,label
0,5.0,❤️❤️❤️❤️❤️,1.0
1,5.0,오래 해주세요🙏,1.0
2,5.0,와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!,1.0
3,5.0,오늘의 메뉴 너무 좋아요. 어쩜 갖가지 닭요리를 그렇게 잘하시죠?!! 이런 리뷰 잘...,1.0
4,1.0,여기서 밥먹고 둘다 장염걸림;;,0.0
...,...,...,...
2196,5.0,음식이 가격대비 양도 많고 맛있는 곳입니다!!!분위기랑 직원들의 반응이 중요하신 분...,1.0
2197,1.0,여기 알바들이 설쳐대는게 아니라면 방송 탔었다고 이리 리뷰가 많은것인가요? 진심 평...,0.0
2198,2.0,음. 웨이팅 십오분했는데 차게 식은 치즈가스가 나왔다. 왕혼가츠도 그냥 쏘쏘.,0.0
2199,5.0,가격은 양에비해서 싸거나 적당하다고 봅니다. 기본 생돈까스는 정말 양이 많슴니다.,1.0


In [63]:
kakao3=kakao[kakao['label']==3].index.tolist()
kakao.drop(kakao3,axis=0,inplace=True)
kakao=kakao.reset_index(drop=True)

,kakao_content,kakao_star,label
0,차돌쭈꾸미 새로생겨서 먹어봤는디 정말맛있어요존맛탱구리구리 오늘도 아주 잘먹구갑니다 ...,5.0,1.0
1,진짜 맛집 근처 다른 해물찜들 먹다가 가게가 보여서 들어와 봤는데 진짜 맛있네요벌써...,5.0,1.0
2,처음 방문했는데 생각보다 많이 달라서 당황했습니다 천안본점에는 밑반찬만 6가지 정도...,1.0,0.0
3,지금껏 먹어본 해물찜과는 정말 다르게 너무 맛있어서 먹고 포장까지 해왔습니다새우찜도...,5.0,1.0
4,생고기 맛 좋아서 추천합니다,5.0,1.0
...,...,...,...
960,음식먹고 기분나쁘기는 처음이네요 일단 사장인지 종업원인지 서비스마인드 아주 저급하네...,1.0,0.0
961,여기 너무맛있음 1인분에 200g씩 주고 찌개도 듬뿍임 ㅋㅋ신선하게 정육점에서 바로...,5.0,1.0
962,환기만 잘 되면완벽한 삼겹살집,5.0,1.0
963,고기도 괜찮고 특히 쌈야채 듬뿍 주셔서 넘 좋네요근데 왜 평점테러를 한 거지,5.0,1.0


In [113]:
rrs=rr[rr['label']==3.5].index.tolist()
rr.drop(rrs,axis=0,inplace=True)
rr=rr.reset_index(drop=True)

In [68]:
hong=hong.drop('user_rating',axis=1)
kakao=kakao.drop('kakao_star',axis=1)
rr=rr.drop('4',axis=1)

In [80]:
#라벨명 바꾸기
hong.rename(columns={'comment':'review'},inplace=True)
kakao.rename(columns={'kakao_content':'review'},inplace=True)
rr.rename(columns={'맛있는데 고기 양이 조금 적을 수도.':'review'},inplace=True)


In [82]:
df=pd.concat([hong,kakao,food_review,thoure,rr],axis=0)
df.head()

,review,label
0,❤️❤️❤️❤️❤️,1.0
1,오래 해주세요🙏,1.0
2,와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!,1.0
3,오늘의 메뉴 너무 좋아요. 어쩜 갖가지 닭요리를 그렇게 잘하시죠?!! 이런 리뷰 잘...,1.0
4,여기서 밥먹고 둘다 장염걸림;;,0.0


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14227 entries, 0 to 14226
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  14227 non-null  object 
 1   label   14227 non-null  float64
dtypes: float64(1), object(1)
memory usage: 222.4+ KB


In [127]:
df.to_csv('리뷰_pre_trained.csv',index=False)

### KoBERT

In [ ]:
!pip install transformers==2.10

In [207]:
!pip install tensorflow-addons

In [208]:
!pip install -q pyyaml h5py

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split

c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\transformers\generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(


In [210]:
df=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\KoBERT 감성분류\리뷰_pre_trained.csv')

In [211]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14227 entries, 0 to 14226
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   review  14227 non-null  object 
 1   label   14227 non-null  float64
dtypes: float64(1), object(1)
memory usage: 222.4+ KB


In [212]:
df['label'].value_counts()

1.0    12279
0.0     1948
Name: label, dtype: int64

In [214]:
df.shape

(14227, 2)

In [215]:
# 타겟레이블 분포 확인 타겟레이블 분포 확인
df['label'].value_counts(normalize=True)
df['label'].value_counts(normalize=True)

1.0    0.863077
0.0    0.136923
Name: label, dtype: float64

In [138]:
#훈련데이터와 검증데이터로 분리

train,test=train_test_split(df,
                            random_state=0,
                            test_size=0.05,
                            stratify=df['label'])

print(train.shape,test.shape)

(13515, 2) (712, 2)


In [142]:
train=train.reset_index(drop=True)
test=test.reset_index(drop=True)

In [143]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

In [216]:
train['label'].value_counts()

1    11664
0     1851
Name: label, dtype: int64

In [217]:
test['label'].value_counts()

1    615
0     97
Name: label, dtype: int64

In [132]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece `_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        self.max_len_single_sentence = self.model_max_length - 2  # take into account special tokens
        self.max_len_sentences_pair = self.model_max_length - 3  # take into account special tokens

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A RoBERTa sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [133]:
#kobert 토크나이즈 임포트

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')


loading file tokenizer_78b3253a26.model from cache at C:\Users\user/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\tokenizer_78b3253a26.model
loading file vocab.txt from cache at C:\Users\user/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\user/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\tokenizer_config.json
loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\config.json
Model config BertConfig {
  "_name_or_path": "monologg/kobert",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "

In [222]:
print(tokenizer.encode("와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!	"))
print(tokenizer.tokenize("와 여길 왜 이제알았죠! 너무맛있고 반찬 하나하나 다 맛있네요!! 자주 갈게요!	"))

[2, 517, 6983, 3298, 5585, 3466, 3742, 6821, 6827, 7275, 5, 1458, 6164, 7141, 5439, 2207, 7395, 4928, 7792, 1562, 1967, 7141, 5703, 5, 5, 3924, 781, 5400, 6999, 5, 3]
['▁', '와', '▁여', '길', '▁왜', '▁이제', '알', '았', '죠', '!', '▁너무', '맛', '있', '고', '▁반', '찬', '▁하나', '하나', '▁다', '▁맛', '있', '네요', '!', '!', '▁자주', '▁갈', '게', '요', '!']


In [223]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 64 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 64
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "review"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

  0%|          | 0/13515 [00:00<?, ?it/s]c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 13515/13515 [00:04<00:00, 3198.57it/s]


In [224]:
test_x,test_y=load_data(test)

100%|██████████| 712/712 [00:00<00:00, 2861.42it/s]


In [156]:
#train_y = tf.expand_dims(train_y, axis=-1)
#test_y = tf.expand_dims(test_y, axis=-1)
train_y = np.expand_dims(train_y, axis=1)
test_y = np.expand_dims(test_y, axis=1)

In [250]:
train_y.shape

(13515,)

### KoBERT 감성분석

In [227]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)

# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')

# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}

loading weights file pytorch_model.bin from cache at C:\Users\user/.cache\huggingface\hub\models--monologg--kobert\snapshots\8ebf2818cfd85570737d31ed8cd7aaa000e7056c\pytorch_model.bin
Loading PyTorch weights from C:\Users\user\.cache\huggingfa

In [256]:
bert_outputs=bert_outputs[1]

In [257]:
bert_outputs

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'tf_bert_model_2')>

In [258]:
bert_outputs.shape

TensorShape([None, 768])

In [259]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa

# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*4, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\tensorflow_addons\optimizers\rectified_adam.py:121: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [262]:
sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

In [263]:
sentiment_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 64)]         0           []                               
                                                                                                  
 input_masks (InputLayer)       [(None, 64)]         0           []                               
                                                                                                  
 input_segment (InputLayer)     [(None, 64)]         0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  92186880    ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_masks[0][0]',      

### 검증

In [271]:
sentiment_model.fit(train_x,train_y,epochs=2,shuffle=True,batch_size=64,validation_data=(test_x,test_y))

Epoch 1/2
212/212 [==============================] - 2490s 12s/step - loss: 0.2269 - accuracy: 0.9217 - val_loss: 0.2376 - val_accuracy: 0.9199
Epoch 2/2
212/212 [==============================] - 2379s 11s/step - loss: 0.1902 - accuracy: 0.9383 - val_loss: 0.2665 - val_accuracy: 0.9143


In [ ]:
#sentiment_model.save('KoBERT_model')
#sentiment_model.save_weights('saveweights')

In [274]:
def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, pad_to_max_length=True)
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [275]:
test_set = predict_load_data(test)

  0%|          | 0/712 [00:00<?, ?it/s]c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 712/712 [00:00<00:00, 2728.20it/s]


In [276]:
preds = sentiment_model.predict(test_set)

23/23 [==============================] - 42s 2s/step


In [277]:
print(preds.shape)
print(y_true.shape)

(712, 1)
(712,)


In [278]:
y_true

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     0
14     1
15     1
16     1
17     1
18     1
19     0
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     0
30     1
31     0
32     1
33     1
34     1
35     1
36     0
37     0
38     0
39     1
40     1
41     1
42     1
43     1
44     1
45     1
46     1
47     1
48     1
49     1
50     1
51     0
52     1
53     1
54     0
55     0
56     1
57     1
58     1
59     1
60     1
61     1
62     1
63     1
64     0
65     1
66     1
67     1
68     0
69     1
70     1
71     1
72     1
73     1
74     1
75     1
76     1
77     1
78     1
79     1
80     1
81     0
82     1
83     1
84     1
85     1
86     1
87     1
88     1
89     1
90     1
91     1
92     1
93     1
94     1
95     1
96     1
97     1
98     0
99     1
100    1
101    1
102    1
103    0
104    0
105    1
106    1
107    1
108    1
109    1
110    1
1

In [279]:
from sklearn.metrics import classification_report
y_true = test['label']
# F1 Score 확인
print(classification_report(y_true, np.round(preds,0)))

              precision    recall  f1-score   support

           0       0.80      0.49      0.61        97
           1       0.92      0.98      0.95       615

    accuracy                           0.91       712
   macro avg       0.86      0.74      0.78       712
weighted avg       0.91      0.91      0.91       712



## 실제 데이터 적용

In [280]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      print("(부정 확률 : %.2f) 부정적인 리뷰입니다." % (1-predict_value))
    elif predict_answer == 1:
      print("(긍정 확률 : %.2f) 긍정적인 리뷰입니다." % predict_value)

In [281]:
review_evaluation_predict("다시는 가고 싶지 않아요")

c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 0s 220ms/step
(부정 확률 : 0.93) 부정적인 리뷰입니다.


In [282]:
review_evaluation_predict("맛은 없는데 식당 내부 분위기는 좋아요.")

c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 0s 194ms/step
(긍정 확률 : 0.98) 긍정적인 리뷰입니다.


In [283]:
review_evaluation_predict("맛있어요!")

1/1 [==============================] - 0s 211ms/step
(긍정 확률 : 0.98) 긍정적인 리뷰입니다.


In [311]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, pad_to_max_length=True)
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

def review_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
      return "(부정 확률 : %.2f) 부정" % (1-predict_value)
    elif predict_answer == 1:
      return "(긍정 확률 : %.2f) 긍정" % predict_value

In [294]:
data=pd.read_csv(r'C:\Users\user\OneDrive\바탕화~1-DESKTOP-R00ORLS-348\캡스톤\data_git\data\diningcode_data\다이닝코드 북촌 리뷰 분리.csv')
data.head()

,상가업소번호,가게명,원래이름,리뷰,분리리뷰,리뷰날짜
0,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...","만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.",2023-3-9
1,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...",칼만두국을 먹었는데 사골베이스에 조금 슴슴한 편인데 맛이 꽤 훌륭하다.,2023-3-9
2,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...",면발은 하늘하늘 부드러운 편이다.,2023-3-9
3,20592276,깡통만두,깡통만두,"만두가 옛날 집에서 빚은 딱 그 맛이어서, 좋았다.칼만두국을 먹었는데 사골베이스에 ...",테이블 간격이 좁고 서버 이모님들이 살짝 불친절한 편이라 온전히 유명세의 맛경험을 ...,2023-3-9
4,20592276,깡통만두,깡통만두,주말에 방문하면 웨이팅이 깁니다. 한시간정도 대기하면 드실 수 있어요. 만두가 깔끔...,주말에 방문하면 웨이팅이 깁니다.,2023-2-19


In [312]:
data['evaluation'] = data['분리리뷰'].apply(review_evaluation_predict)

c:\Users\user\anaconda3\envs\capstone\Lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


1/1 [==============================] - 0s 165ms/step


In [314]:
data.to_csv('test_review_predict.csv',index=False)

In [315]:
data.to_excel('test_review_predict.xlsx',index=False)